# A program to generate statistics on Absolute Radio

The program monitors the weekday show (9am - 6pm) on absolute radio and will:

1. 2. generate statistics on the number of songs played
3. (in future) keep a tally of genres so that we can make some interesting plots in plotly

There are a few things that must be implemented

1. Getting data to plotly
2. Running program only on specific days of the week
3. Genres

In [1]:
'''
Created on 15 Jan 2017

@author: afunn
'''
print("Absolute Radio - Did they repeat?")

Absolute Radio - Did they repeat?


### Import the required libraries

In [2]:
from lxml import html  # for scraping web information
import requests
from threading import Timer  # for timing
from collections import Counter  # for counting list
import time
import plotly
from datetime import datetime
import pandas as pd
plotly.tools.set_credentials_file(username='benisme', api_key='clBZNegtB5pqHJZXRnnP')
plotly.__version__

'2.0.8'

### Definitions
#### Create the global variables that are used throughout the program
It is generally not best practice to do this, but because we are using the timer, it is quite a difficult problem to get around and this is an easy solution

#### Other Definitions
We use a number of lists to keep track of what's happening with the songs and counts

In [3]:
# Global variables (Very bad to do this :( but its easy)
global CurrentRepeatList
global CurrentRepeatListLength
CurrentRepeatList = []
CurrentRepeatListLength = 0

### Here we write the code to strip the data from the absolute radio webpage

This is needed at the start of the program so that the starting state of the program is defined.
We place this into three arrays, *artists*, *songs* and *times*

In [4]:
# Get the data from absolute radio

page = requests.get('https://absoluteradio.co.uk/absolute-radio/music/')
tree = html.fromstring(page.content)

# This will create a list of artists
artists = tree.xpath('//p[@class="song-artist"]/text()')
# This will create a list of songs
songs = tree.xpath('//p[@class="song-title"]/a/text()')
# This will create a list of the times of the songs
times = tree.xpath('//div[@class="song-inner"]/time/text()')

We then print these to the console, just so that we know things are working

In [5]:
print(artists)
print(songs)
print(times)

['Stevie Wonder', 'Talking Heads', 'Mumford and Sons', 'The Cure', 'Mark Ronson', 'The Knack', 'The Verve', 'Aerosmith/Run DMC', 'Maroon 5', 'Arctic Monkeys', 'R.E.M.', 'Adele', 'The Smiths', 'George Ezra']
['Higher Ground', 'Road To Nowhere', 'I Will Wait', "Friday I'm In Love", 'Valerie', 'My Sharona', 'Lucky Man', 'Walk This Way', 'She Will Be Loved', 'Snap Out Of It', 'Near Wild Heaven', 'Skyfall', 'How Soon Is Now?', 'Budapest']
['5.13pm', '5.11pm', '5.06pm', '5.03pm', '4.59pm', '4.55pm', '4.44pm', '4.34pm', '4.30pm', '4.27pm', '4.24pm', '4.11pm', '4.09pm', '4.05pm']


Here we will change the times in the *ArtistsSongsTimesCount* list to standard format

In [6]:
x = 0
print(times)

for item in times:
    time_string = item
    #plus_string = '2009-11-29'
    plus_string = datetime.today().strftime("%Y-%m-%d")
    newformat = '%Y-%m-%d %I.%M%p'
    format1 = '%Y-%m-%d %I:%M %p'
    total_string = plus_string + " " + time_string
    my_date = datetime.strptime(total_string, newformat)
    times[x] = my_date.strftime(format1)
    x = x+1
    
print(times)

['5.13pm', '5.11pm', '5.06pm', '5.03pm', '4.59pm', '4.55pm', '4.44pm', '4.34pm', '4.30pm', '4.27pm', '4.24pm', '4.11pm', '4.09pm', '4.05pm']
['2017-05-13 05:13 PM', '2017-05-13 05:11 PM', '2017-05-13 05:06 PM', '2017-05-13 05:03 PM', '2017-05-13 04:59 PM', '2017-05-13 04:55 PM', '2017-05-13 04:44 PM', '2017-05-13 04:34 PM', '2017-05-13 04:30 PM', '2017-05-13 04:27 PM', '2017-05-13 04:24 PM', '2017-05-13 04:11 PM', '2017-05-13 04:09 PM', '2017-05-13 04:05 PM']


And we define two variable lists based on the length of the number of songs that was returned.
Note: i'm not sure what emailnotification does any more

In [7]:
# Main list definitions
ArtistsSongsTimesCount = [artists, songs, times, [0] * len(artists)]

EmailNotification = [0] * len(artists) # note this can be deleted

for x in range(0,(len(ArtistsSongsTimesCount[3]))):
    ArtistsSongsTimesCount[3][x] = x + 1
    
print(ArtistsSongsTimesCount[3])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]


Here we write to a file at the start of the program all of the songs which have been played so far. This will become our master file which we will use to generated statistics.

In [8]:
OutputFinal = []

f = open('Today_Songlist', 'a')

f.write('Artists;Songs;Times;Genres\n')

for i in range(0, (len(ArtistsSongsTimesCount[0]) - 1)):
    ReadyForOutput = ""
    for list in ArtistsSongsTimesCount:
        ReadyForOutput = ReadyForOutput + str(list[i]) + ';'
    ReadyForOutput = ReadyForOutput + '\n'
    OutputFinal.append(ReadyForOutput)

for item in OutputFinal:
    f.write(item)
    
f.close()

Testing that the file reads

In [9]:
f = open('Today_Songlist', 'r')
hello = f.read()
f.close()

print(hello)


Artists;Songs;Times;Genres
Stevie Wonder;Higher Ground;2017-05-13 05:13 PM;1;
Talking Heads;Road To Nowhere;2017-05-13 05:11 PM;2;
Mumford and Sons;I Will Wait;2017-05-13 05:06 PM;3;
The Cure;Friday I'm In Love;2017-05-13 05:03 PM;4;
Mark Ronson;Valerie;2017-05-13 04:59 PM;5;
The Knack;My Sharona;2017-05-13 04:55 PM;6;
The Verve;Lucky Man;2017-05-13 04:44 PM;7;
Aerosmith/Run DMC;Walk This Way;2017-05-13 04:34 PM;8;
Maroon 5;She Will Be Loved;2017-05-13 04:30 PM;9;
Arctic Monkeys;Snap Out Of It;2017-05-13 04:27 PM;10;
R.E.M.;Near Wild Heaven;2017-05-13 04:24 PM;11;
Adele;Skyfall;2017-05-13 04:11 PM;12;
The Smiths;How Soon Is Now?;2017-05-13 04:09 PM;13;



And then we print some stuff into the console to check things are working

In [10]:
print(len(artists))
print(ArtistsSongsTimesCount)

14
[['Stevie Wonder', 'Talking Heads', 'Mumford and Sons', 'The Cure', 'Mark Ronson', 'The Knack', 'The Verve', 'Aerosmith/Run DMC', 'Maroon 5', 'Arctic Monkeys', 'R.E.M.', 'Adele', 'The Smiths', 'George Ezra'], ['Higher Ground', 'Road To Nowhere', 'I Will Wait', "Friday I'm In Love", 'Valerie', 'My Sharona', 'Lucky Man', 'Walk This Way', 'She Will Be Loved', 'Snap Out Of It', 'Near Wild Heaven', 'Skyfall', 'How Soon Is Now?', 'Budapest'], ['2017-05-13 05:13 PM', '2017-05-13 05:11 PM', '2017-05-13 05:06 PM', '2017-05-13 05:03 PM', '2017-05-13 04:59 PM', '2017-05-13 04:55 PM', '2017-05-13 04:44 PM', '2017-05-13 04:34 PM', '2017-05-13 04:30 PM', '2017-05-13 04:27 PM', '2017-05-13 04:24 PM', '2017-05-13 04:11 PM', '2017-05-13 04:09 PM', '2017-05-13 04:05 PM'], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]]


## Plotly

In [11]:
import plotly.dashboard_objs as dashboard

import IPython.display
from IPython.display import Image

my_dboard = dashboard.Dashboard()
my_dboard.get_preview()


In [12]:
import plotly.plotly as py
import plotly.graph_objs as go

import pandas as pd

headers = ['Artists', 'Songs', 'DateTime', 'Genres']
dtypes = {'Artists': 'str', 'Songs': 'str', 'DateTime': 'str', 'Genres': 'str'}
parse_dates = ['DateTime']

df = pd.read_csv('Today_Songlist', sep=';', header=None, names=headers, dtype=dtypes, parse_dates=parse_dates)

#df = pd.read_csv('Today_Songlist', sep=';')

print(df)


trace_high = go.Scatter(
                x=df.DateTime,
                y=df['Genres'],
                mode='markers',
                text=df['Songs'],
                name = "Genres",
                line = dict(color = '#17BECF'),
                opacity = 0.8)

data = [trace_high]

layout = dict(
    title = "First trial at Plotly",
    xaxis = dict(
        range = ['2017-05-12','2017-05-15'])
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename = "Manually Set Range")



              Artists               Songs             DateTime  Genres
0             Artists               Songs                Times  Genres
1       Stevie Wonder       Higher Ground  2017-05-13 05:13 PM       1
2       Talking Heads     Road To Nowhere  2017-05-13 05:11 PM       2
3    Mumford and Sons         I Will Wait  2017-05-13 05:06 PM       3
4            The Cure  Friday I'm In Love  2017-05-13 05:03 PM       4
5         Mark Ronson             Valerie  2017-05-13 04:59 PM       5
6           The Knack          My Sharona  2017-05-13 04:55 PM       6
7           The Verve           Lucky Man  2017-05-13 04:44 PM       7
8   Aerosmith/Run DMC       Walk This Way  2017-05-13 04:34 PM       8
9            Maroon 5   She Will Be Loved  2017-05-13 04:30 PM       9
10     Arctic Monkeys      Snap Out Of It  2017-05-13 04:27 PM      10
11             R.E.M.    Near Wild Heaven  2017-05-13 04:24 PM      11
12              Adele             Skyfall  2017-05-13 04:11 PM      12
13    

### The master function

Here we define the master function which will be called by the timer, this will
1. Call the *Retrieve_TimesArtistsSongs()* function 
2. If the *Retrieve_TimesArtistsSongs()* returns true, indicating a new song has been played, the master function will
  * Check whether it is a repeat: It does this by running *UpdateCount()*
  * *Update Count* Returns the current list of repeated songs, and the number of songs played
  * If the length of the current list of repeated songs has changed from the last loop, then actions are generated
  
3. Return values back to the timer

In [13]:
# start of master function definition
def update_song_list_count_and_email():
    global CurrentRepeatList
    global CurrentRepeatListLength

    NewSongBoolean = False
    RepeatBoolean = False
    TotalSongs = 0

    NewSongBoolean = Retrieve_TimesArtistsSongs()
    print("")
    print("NewSongBoolean = ", NewSongBoolean)
    # change this to a case statement later on
    if NewSongBoolean == True:
        print("Updating List")

    else:
        # UpdateCount2()
        print("Not updating this time around")

    print("CurrentRepeatList = ", CurrentRepeatList)
    print("len(CurrentRepeatList) = ", len(CurrentRepeatList))

    CurrentRepeatListLength = len(CurrentRepeatList)

    # If the update count returns a bigger value then do something

    print("")
    print("ArtistsSongsTimesCount = ", ArtistsSongsTimesCount)
    print("")
   
    # Function check to see if count is > EmailNotificaiton list

    return CurrentRepeatListLength, TotalSongs

## The *Retrieve_TimesArtistsSongs* function

What this function is doing is:
1. Getting the current list of Artists, songs, times from the absolute radio website
2. Comparing the songs list to the existing songs list, and if there is a difference
  1. It inserts the new song into the existing array
  2. It returns the value 'true'

In [14]:
def Retrieve_TimesArtistsSongs():
    page = requests.get('https://absoluteradio.co.uk/absolute-radio/music/')
    tree = html.fromstring(page.content)
    # This will create a list of artists
    artists_B = tree.xpath('//p[@class="song-artist"]/text()')
    # This will create a list of songs
    songs_B = tree.xpath('//p[@class="song-title"]/a/text()')
    # This will create a list of the times of the songs
    times_B = tree.xpath('//div[@class="song-inner"]/time/text()')

    if songs_B[0] != ArtistsSongsTimesCount[1][0]: # i.e. compare the newly created songs array to the existing songs array
        ArtistsSongsTimesCount[0].insert(0, artists_B[0])
        ArtistsSongsTimesCount[1].insert(0, songs_B[0])
        ArtistsSongsTimesCount[2].insert(0, times_B[0])
        ArtistsSongsTimesCount[3].insert(0, 0)
        return True

    else:
        return False

Not sure what this any more - does this code need to be deleted? 

In [ ]:
def UpdateCount2():
    for x in range(len(ArtistsSongsTimesCount[0])):
        print(x)
    # if c == 3:
    #    continue
    #print (c)

## Timer Function

In [ ]:
# Creates a timer function running for 30s, and running function get songs

starttime = time.time()
while True:
    CurrentRepeatList2, TotalSongs2 = update_song_list_count_and_email()
    # Code needed here to
    # pass back whether there has been a repeat and the total number of songs.
    # also check if need to re-tweet at someone
    time.sleep(30.0 - ((time.time() - starttime) % 30.0))


NewSongBoolean =  False
Not updating this time around
CurrentRepeatList =  []
len(CurrentRepeatList) =  0

ArtistsSongsTimesCount =  [['Stevie Wonder', 'Talking Heads', 'Mumford and Sons', 'The Cure', 'Mark Ronson', 'The Knack', 'The Verve', 'Aerosmith/Run DMC', 'Maroon 5', 'Arctic Monkeys', 'R.E.M.', 'Adele', 'The Smiths', 'George Ezra'], ['Higher Ground', 'Road To Nowhere', 'I Will Wait', "Friday I'm In Love", 'Valerie', 'My Sharona', 'Lucky Man', 'Walk This Way', 'She Will Be Loved', 'Snap Out Of It', 'Near Wild Heaven', 'Skyfall', 'How Soon Is Now?', 'Budapest'], ['2017-05-13 05:13 PM', '2017-05-13 05:11 PM', '2017-05-13 05:06 PM', '2017-05-13 05:03 PM', '2017-05-13 04:59 PM', '2017-05-13 04:55 PM', '2017-05-13 04:44 PM', '2017-05-13 04:34 PM', '2017-05-13 04:30 PM', '2017-05-13 04:27 PM', '2017-05-13 04:24 PM', '2017-05-13 04:11 PM', '2017-05-13 04:09 PM', '2017-05-13 04:05 PM'], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]]


NewSongBoolean =  False
Not updating this time aro

Ideas for next steps

In [ ]:
"""    
At end of day (1) tweet about the number of songs and repeats and (2) write day's output to file'

I can have 2 seperate functions (1) that tweets to twitter (2) one that checks twitter



Settings file: 

days of the week to run

start time

stop time


Generated file:

How many full days monitored

How many repeats
"""